In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

In [30]:
from collections import OrderedDict
fields = OrderedDict([('height', 170), ('weight', 70), ('age', 20)])

for f in fields:
    print(f)

height
weight
age


In [35]:
from collections import OrderedDict
fields = OrderedDict({'height': 170, 'weight': 70, 'age': 20})

for f in fields:
    print(f)

age
height
weight


In [37]:
dependencies = OrderedDict([('signame', 'checksum'),
                                ('blocksize', 'checksum'),
                                ('checksum', 'initvalue'),
                                ('initvalue', 'adczero'),
                                ('adczero', 'adcres'),
                                ('adcres', 'adcgain'),
                                ('units', 'adcgain'),
                                ('baseline', 'adcgain'),
                                ('adcgain', 'fmt'),
                                ('byteoffset', 'fmt'),
                                ('skew', 'fmt'),
                                ('sampsperframe', 'fmt')
                               ])

for d in dependencies:
    print(d)
    print(dependencies[d])

signame
checksum
blocksize
checksum
checksum
initvalue
initvalue
adczero
adczero
adcres
adcres
adcgain
units
adcgain
baseline
adcgain
adcgain
fmt
byteoffset
fmt
skew
fmt
sampsperframe
fmt


In [7]:
fields = {'height': 170, 'weight': 70, 'age': 20}

for i in reversed(list(fields)):
    print(i)

weight
height
age


SyntaxError: invalid syntax (<ipython-input-17-8c8a4646591e>, line 1)

In [48]:
import numpy as np
a=np.zeros([10,3])
m, n = np.shape(a)


In [4]:
WFDBfields = ['height', 'weight', 'age']

final = dict.fromkeys(WFDBfields, [])
final

{'age': [], 'height': [], 'weight': []}

In [18]:
help(dict.fromkeys)

Help on built-in function fromkeys:

fromkeys(iterable, value=None, /) method of builtins.type instance
    Returns a new dict with keys from iterable and values equal to value.



In [20]:
for i in final:
    print(i)

age
weight
height


In [5]:
for i in reversed(WFDBfields):
    print(i)

age
weight
height


In [57]:
a=[{'recname': 'wave1'},{'siglen': 500},{'fs': 125}]

In [62]:
b=[i.keys() for i in a]
b

[dict_keys(['recname']), dict_keys(['siglen']), dict_keys(['fs'])]

In [3]:
# A field of the WFDB header file
from collections import OrderedDict

class WFDBfield():
    
    def __init__(self, speclist):
        # Name
        self.name = speclist[0] 
        
        # Data types the field can take
        self.allowedtypes = speclist[1]
        
        # The text delimiter that preceeds the field
        self.delimiter = speclist[2]
        
        # The required/dependent field which must also be present
        self.field_req = speclist[3]

        # Whether the field is mandatory specified by the WFDB guidelines
        self.is_req = speclist[4]
        
        # Whether the field is mandatory for writing (extra rules enforced by this library).
        # Being required for writing is not the same as the user having to specify. There are defaults. 
        self.write_req = speclist[5]

In [18]:
WFDBfields = [
    
    # record specification fields
    OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True])),
                 ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False])),
                 ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True])),
                 ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True])),
                 ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False])),
                 ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False])),
                 ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True])),
                 ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False])),
                 ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False]))]),
    
    # signal specification fields
    OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True])),
                 ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True])),
                 ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                 ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                 ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                 ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                 ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                 ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                 ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                 ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                 ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                 ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                 ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                 ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))]),
    
    # segment specification fields
    OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                 ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))]),
    
    # comment fields
    OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])
]

In [5]:
f1=WFDBfields[0]

In [11]:
f2=OrderedDict(reversed(list(f1.items())))

In [12]:
f2

OrderedDict([('basedate', <__main__.WFDBfield at 0x7fc2ee639400>),
             ('basetime', <__main__.WFDBfield at 0x7fc2ee6393c8>),
             ('siglen', <__main__.WFDBfield at 0x7fc2ee639390>),
             ('basecounter', <__main__.WFDBfield at 0x7fc2ee639358>),
             ('counterfreq', <__main__.WFDBfield at 0x7fc2ee639320>),
             ('fs', <__main__.WFDBfield at 0x7fc2ee6392e8>),
             ('nsig', <__main__.WFDBfield at 0x7fc2ee6392b0>),
             ('nseg', <__main__.WFDBfield at 0x7fc2ee639278>),
             ('recordname', <__main__.WFDBfield at 0x7fc2ee639240>)])

In [23]:
dir(WFDBfields[0])

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'move_to_end',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [28]:
help(WFDBfields[0].setdefault)

Help on built-in function setdefault:

setdefault(...) method of collections.OrderedDict instance
    od.setdefault(k[,d]) -> od.get(k,d), also set od[k]=d if k not in od



In [76]:
bd={'name':'Jack'}
bd['strength']=10
bd

{'name': 'Jack', 'strength': 10}

In [79]:
if 'strength' in bd:
    print('ya')

ya
